<a href="https://colab.research.google.com/github/parasgulati8/Web-Scraper/blob/master/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import glob
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup as bs
# import urllib3
import requests
# import json
# from urllib.request import urlopen, urlretrieve, quote
# from openpyxl import Workbook
import re

In [0]:
# Property Summary Page
def get_property_summary(index):
    print('fetching Property Summary for ', index)
    url = 'http://www.padctn.org/prc/property/' + str(index) + '/card/1'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    lists = soup.find_all('ul', attrs={'class':'att'})
    table = dict()
    for i in range(len(lists)):
      lists[i] = lists[i].find_all('li')
      lists[i] = [item.text for item in lists[i]]
      lists[i] = [lists[i][j].split(': ') for j in range(len(lists[i]))]
      for pair in lists[i]:
        key , value = pair
        table[key] = value
    
    index = 1
    summary = pd.DataFrame(table, index=[index])
    return summary

# Historical data
def get_historical_data(index):
    print('fetching historical data for ', index)
    url = 'http://www.padctn.org/prc/property/' + str(index) + '/card/1/historical'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    parcel = soup.h2.text
    parcel = ' '.join(parcel.split()[6:])
    dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                          attrs={"class":"table table-condensed table-bordered"})
    sales_history, previous_appraisals = dfs[0], dfs[1]
    sales_history['Map & Parcel'] = [parcel]*len(sales_history)
    sales_history['Property #'] = [index]*len(sales_history)
    previous_appraisals['Map & Parcel'] = [parcel]*len(previous_appraisals)
    previous_appraisals['Property #'] = [index]*len(previous_appraisals)
    return sales_history, previous_appraisals, parcel
                        

# Building and Improvement Details Page
def get_building_improvement(index, parcel):
    print('fetching building improvement for ', index)
    url = 'http://www.padctn.org/prc/property/' + str(index) + '/card/1/interior'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    lists = (soup.find_all('ul', attrs={'class':'att'}))
    table = dict()
    for i in range(len(lists)):
      lists[i] = lists[i].find_all('li')
      lists[i] = [item.text for item in lists[i]]
      lists[i] = [lists[i][j].split(': ') for j in range(len(lists[i]))]
      for pair in lists[i]:
        key , value = pair
        table[key] = value
    
    temp_table = pd.DataFrame(table, index=[0])
    temp_table['Map & Parcel'] = [parcel]
    dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text)
    building_footage, building_features = dfs[0], dfs[1]
    building_features['Map & Parcel'] = [parcel]*(1 if len(building_features) == 0 else len(building_features))
    building_footage['Map & Parcel'] = [parcel]*len(building_footage)
    
    merged = pd.merge(building_footage, building_features, on='Map & Parcel')
    building_improvement = pd.merge(merged, temp_table, on='Map & Parcel')
    return building_improvement

def terminate(index):
    for i in range(index+1,index+6):
        temp_history, temp_appraisal = get_historical_data(i)
        if (np.any(temp_history) or np.any(temp_appraisal)):                  
            return False
    return True

property_summary = get_property_summary(1)
sales_history, previous_appraisals, parcel = get_historical_data(1)
building_improvement = get_building_improvement(1, parcel)


index = 2
patience = 0
while True:
      # Property Summary Page
      try:
        _property_summary =  get_property_summary(index)
        if np.any(_property_summary):    
          property_summary = pd.concat((property_summary, _property_summary))
      except:
        print('Exception occured in fetching Property Summary page for index', index)
      
      # Historical data
      try:
        _sales_history, _previous_appraisals, parcel = get_historical_data(index)
        if np.any(_sales_history):
            sales_history = pd.concat((sales_history, _sales_history))
        if np.any(_previous_appraisals):
            previous_appraisals = pd.concat((previous_appraisals, _previous_appraisals))
      except:
          print('Exception occured in fetching Sales History page for index', index)

      # Building and Improvements Details
      try:
        _building_improvement = get_building_improvement(index, parcel)
        if np.any(_building_improvement):
            building_improvement = pd.concat((building_improvement, _building_improvement))
      except:
        print('Exception occured in fetching Building Improvement page for index', index)  
      
      if not (np.any(_sales_history) or np.any(_previous_appraisals)):
          try:
            end  = terminate(index)
            print ('Checking to terminate')  
            if end == True:
                print('Scraping Ends. Last propert # scraped is', index)
                print (_property_summary, _sales_history, _previous_appraisals, _building_improvement)
                break
          except:
            print('Exception occured in checking termination for index', index)      
            break
          
      print(index)
      index = index+1
      if index == 5:
          break 

index = 2
patience = 0
while True:
      # Property Summary Page
      try:
        _property_summary =  get_property_summary(index)
        if np.any(_property_summary):    
          property_summary = pd.concat((property_summary, _property_summary))
      except:
        print('Exception occured in fetching Property Summary page for index', index)
      
      # Historical data
      try:
        _sales_history, _previous_appraisals, parcel = get_historical_data(index)
        if np.any(_sales_history):
            sales_history = pd.concat((sales_history, _sales_history))
        if np.any(_previous_appraisals):
            previous_appraisals = pd.concat((previous_appraisals, _previous_appraisals))
      except:
          print('Exception occured in fetching Sales History page for index', index)

      # Building and Improvements Details
      try:
        _building_improvement = get_building_improvement(index, parcel)
        if np.any(_building_improvement):
            building_improvement = pd.concat((building_improvement, _building_improvement))
      except:
        print('Exception occured in fetching Building Improvement page for index', index)  
      
      if not (np.any(_sales_history) or np.any(_previous_appraisals)):
          try:
            end  = terminate(index)
            print ('Checking to terminate')  
            if end == True:
                print('Scraping Ends. Last propert # scraped is', index)
                print (_property_summary, _sales_history, _previous_appraisals, _building_improvement)
                break
          except:
            print('Exception occured in checking termination for index', index)      
            break
          
      print(index)
      index = index+1
#      if index == 5:
#          break 
      if index %1000:
        property_summary.to_csv('Property Summary.csv')
        sales_history.to_csv('Sales History.csv')
        previous_appraisals.to_csv('Previous Appraisals.csv')
        building_improvement.to_csv('Building Improvement.csv')

property_summary.to_csv('Property Summary.csv')
sales_history.to_csv('Sales History.csv')
previous_appraisals.to_csv('Previous Appraisals.csv')
building_improvement.to_csv('Building Improvement.csv')


5